# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [8]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment
openai = OpenAI()



In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

system_prompt = """You are a technical assistant answering technical question and providing detailed explanations that goes with the answer. Format the response in the following manner
Question: What is 2+2?
Answer: 4
Explanation: 2 + 2 = 4  because if you have 2, etc...
 """

user_prompt = """ Provide a detailed response to the question I provide you. """ + question

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [9]:
# Get gpt-4o-mini to answer, with streaming

stream = openai.chat.completions.create(messages=messages, model="gpt-4o-mini",stream=True)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

Question: What does the code `yield from {book.get("author") for book in books if book.get("author")}` do and why?

Answer: The code is intended to yield unique authors from a collection of books.

Explanation: 
1. **Context of the Code**:
   - The code snippet is likely part of a generator function in Python that uses the `yield` statement. This function allows for iteration over results without storing all the results in memory, which is particularly useful for large datasets.

2. **Understanding Each Part**:
   - `books` is expected to be a list (or any iterable) of dictionaries, where each dictionary represents a book.
   - `book.get("author")` accesses the value associated with the key `"author"` in each book dictionary. 
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. This constructs a set of authors, filtering out any `None` (or falsy) values. The `if book.get("author")` part ensures that only books with a defined author are considered.

3. **Using `yield from`**:
   - `yield from` is a statement that delegates the yielding of values to another iterable. In this case, it will yield each author obtained from the set comprehension.
   - Since a set does not allow duplicate values, this effectively ensures that each author is yielded only once, thereby providing unique authors from the list of books.

4. **Why Use This Code**:
   - This approach is notably efficient. By using a generator, the code avoids the need to maintain a large list or array in memory, yielding authors one at a time as they are requested. 
   - It is a clean and Pythonic way to gather unique values while also handling the possibility of missing authors gracefully.

5. **Example**:
   - Suppose we have a `books` list like this:
     python
     books = [
         {"title": "Book A", "author": "Author 1"},
         {"title": "Book B", "author": "Author 2"},
         {"title": "Book C", "author": "Author 1"},
         {"title": "Book D", "author": None}
     ]
     
   - The resulting set comprehension will produce `{"Author 1", "Author 2"}`, and calling the generator would yield each unique author in that set.

In summary, this code snippet is designed to efficiently provide a unique list of authors from a collection of books, yielding the authors as they are requested, ensuring memory efficiency and clarity.

In [14]:
# Get Llama 3.2 to answer

import ollama


response = ollama.chat(model=MODEL_LLAMA, messages=messages)
display_handle = display(Markdown(""), display_id=True)
update_display(Markdown(response["message"]["content"]), display_id=display_handle.display_id)

**Code Explanation:**

The given code is a Python expression that uses a combination of generators, dictionary methods, and conditional statements to fetch the authors of books from a list of books. Here's a breakdown of what each part does:

1. `{book.get("author") for book in books if book.get("author")}`:
   - This is a generator expression, which is a compact way to create iterators.
   - `for book in books` iterates over each book in the list `books`.
   - `if book.get("author")` filters out books that don't have an "author" key. The `get()` method returns the value associated with a given key if it exists, otherwise, it returns `None`. In this context, it's used to check if a book has an author without throwing a KeyError.
   - `book.get("author")` extracts the author from each book that passes the previous condition.

2. `yield from {book.get("author") for book in books if book.get("author")}`:
   - This line uses the `yield from` keyword, which is available in Python 3.3 and later versions.
   - The `yield from` statement takes an iterable (in this case, another generator expression) as an argument and yields each item from that iterable.
   - By wrapping the generator expression in `yield from`, we can create a new iterator that "delegates" to the inner generator expression. This is useful when you want to iterate over a sequence of items without having to explicitly create an intermediate list or other data structure.

**What does this code do?**

In summary, this code generates an iterator that yields authors for each book in the `books` list, skipping books without an author. The resulting iterator can be used in any context where you want to iterate over a sequence of authors, such as when generating a report or creating a dataset.

**Example usage:**
```python
# Assuming 'books' is a list of dictionaries with 'author' keys
books = [
    {"title": "Book 1", "author": "John Doe"},
    {"title": "Book 2"},
    {"title": "Book 3", "author": "Jane Smith"}
]

authors = yield from {book.get("author") for book in books if book.get("author")}
for author in authors:
    print(author)
```
Output:
```
John Doe
Jane Smith
```